In [1]:
import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

client = openai.Client()

## Cria o assistant

In [2]:
assitant = client.beta.assistants.create(
    name="Tutor de Matemática da Asimov",
    instructions='Você é um tutor pessoal de matemática da empresa Asimov. \
        Escreva e execute códigos para responder as perguntas de matemática que lhe forem passadas.',
    tools=[{'type': 'code_interpreter'}],
    model='gpt-3.5-turbo-0125'
)

## Cria uma thread

In [3]:
thread = client.beta.threads.create()

## Adiciona mensagem a thread criada

In [4]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role='user',
    content='Se eu jogar um dado honesto 1000 vezes, qual é a probabilidade de eu obter exatamente 150 vezes o número 6? Resolva com um código'
)

## Roda a thread no assistant

In [5]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assitant.id,
    instructions='O nome do usuário é Adriano Soares e ele é um usuário Premium.'
)

## Aguarda a thread rodar

In [6]:
import time

while run.status in ['queued', 'in_progress', 'cancelling']:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )


In [7]:
run.status

'completed'

## Verifica a resposta

In [8]:
if run.status == 'completed':
    mensagens = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(mensagens)
else:
    print('Errro', run.status)

SyncCursorPage[Message](data=[Message(id='msg_SGnJdpFRsxIk51XeTGuw96ix', assistant_id='asst_wAeXszZJS36MbLWkn6SsrhaK', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='A probabilidade de obter exatamente 150 vezes o número 6 ao jogar um dado honesto 1000 vezes é aproximadamente 0.0126 ou 1.26%.'), type='text')], created_at=1724163156, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_9urS5vHLpAY2LD4lKULrBkfo', status=None, thread_id='thread_lKd6nX6f7wA1pC7SF2WnqZWj'), Message(id='msg_gWSO8pPkVHwdEny0UkBI5MGQ', assistant_id='asst_wAeXszZJS36MbLWkn6SsrhaK', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Para calcular a probabilidade de obter exatamente 150 vezes o número 6 ao jogar um dado honesto 1000 vezes, podemos utilizar a distribuição binomial. A fórmula da distribuição binomial é dada por:\n\n\\[\nP(X = k) = \\binom{n}{k} \\time

In [9]:
print(mensagens.data[0].content[0].text.value)

A probabilidade de obter exatamente 150 vezes o número 6 ao jogar um dado honesto 1000 vezes é aproximadamente 0.0126 ou 1.26%.


## Analisando os passos do modelo

In [10]:
run_steps = client.beta.threads.runs.steps.list(
  thread_id=thread.id,
  run_id=run.id
)

In [11]:
for step in run_steps.data:
    print('=== Step:', step.step_details.type)
    if step.step_details.type == 'tool_calls':
        for tool_call in step.step_details.tool_calls:
            print('-----')
            print(tool_call.code_interpreter.input)
            print('-----')
            print('Result')
            print(tool_call.code_interpreter.outputs[0].logs)
    

=== Step: message_creation
=== Step: tool_calls
-----
from math import comb

# Definindo os parâmetros
n = 1000  # total de tentativas
k = 150   # número de sucessos desejados
p = 1/6   # probabilidade de sucesso em uma única tentativa

# Calculando a probabilidade usando a fórmula da distribuição binomial
probabilidade = comb(n, k) * (p**k) * ((1-p)**(n-k))
probabilidade
-----
Result


IndexError: list index out of range

In [ ]:
for step in run_steps.data[::-1]:
    print('\n=== Step:', step.step_details.type)
    if step.step_details.type == 'tool_calls':
        for tool_call in step.step_details.tool_calls:
            print('-----')
            print(tool_call.code_interpreter.input)
            print('-----')
            print('Result')
            print(tool_call.code_interpreter.outputs[0].logs)
    if step.step_details.type == 'message_creation':
        message = client.beta.threads.messages.retrieve(
            thread_id=thread.id,
            message_id=step.step_details.message_creation.message_id
        )
        print(message.content[0].text.value)
    


=== Step: message_creation
Para calcular a probabilidade de obter exatamente 150 vezes o número 6 ao lançar um dado honesto 1000 vezes, podemos usar a distribuição binomial. A fórmula para a probabilidade em uma distribuição binomial é dada por:

\[ P(X = k) = \binom{n}{k} \times p^k \times q^{n-k} \]

onde:
- \( n \) é o número total de tentativas (lançamentos do dado),
- \( k \) é o número de sucessos desejado (número 6),
- \( p \) é a probabilidade de sucesso em uma única tentativa,
- \( q \) é a probabilidade de fracasso em uma única tentativa (1 - p).

Neste caso, \( n = 1000 \) (número total de lançamentos do dado), \( k = 150 \) (número de vezes em que queremos obter o número 6), \( p = \frac{1}{6} \) (probabilidade de obter o número 6 em um único lançamento) e \( q = 1 - p \).

Vamos calcular essa probabilidade com um código Python:

=== Step: tool_calls
-----
from math import comb

# Definindo os parâmetros
n = 1000  # número total de lançamentos do dado
k = 150   # número de